In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [4]:
import nltk
nltk.download('punkt_tab') # download 'punkt_tab' resource
nltk.download('wordnet') #download 'wordnet' resource
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
df_train['text'] = [word_tokenize(entry) for entry in df_train['text']] # Tokenize

In [6]:
df_test['text'] = [word_tokenize(entry) for entry in df_test['text']]

NOTE: CREATE A FUNCTION TO DO THIS FOR BOTH TEST AND TRAIN CSV

In [7]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df_train['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag below will provide the 'tag' if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    df_train.loc[index,'text_final'] = str(Final_words)

    #Est. time to run 24s

In [8]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df_test['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    df_test.loc[index,'text_final'] = str(Final_words)

#Est. time to run 12s

In [11]:
df_test['target'] = ''

In [12]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
x = Tfidf_vect.fit_transform(df_train['text_final'])
x_test = Tfidf_vect.transform(df_test['text_final'])
y = df_train['target']

In [34]:
#Split into validation and training sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)#ts = 0.2

In [35]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

svm_model = SVC(kernel='rbf', C=1.0,  class_weight='balanced', random_state=42) #default 'rbf' kernal
svm_model.fit(x_train, y_train)

pred = svm_model.predict(x_val)

accuracy_svm = accuracy_score(y_val, pred)

print(f"Valid Accuracy: {accuracy_svm:.2f}")

print("\nClassification:")
print(classification_report(y_val, pred))

print("\nConfusion:")
print(confusion_matrix(y_val, pred))

Valid Accuracy: 0.80

Classification:
              precision    recall  f1-score   support

           0       0.80      0.86      0.83       874
           1       0.79      0.70      0.75       649

    accuracy                           0.80      1523
   macro avg       0.80      0.78      0.79      1523
weighted avg       0.80      0.80      0.79      1523


Confusion:
[[755 119]
 [192 457]]


In [16]:
#test_pred = svm_model.predict(x_test)
#df_test['target'] = test_pred

In [ ]:
#sub_file = df_test[['id', 'target']]
#sub_file.to_csv('submission.csv', index=False)

In [ ]:
#print(sub_file.head(10))

NameError: name 'sub_file' is not defined